
***Uncomment the line to install newspaper3k first***



In [46]:
# ! pip install newspaper3k

In [2]:
# importing necessary libraries
from bs4 import BeautifulSoup
import requests
import urllib
import pandas as pd
from newspaper import Article
import pickle
import re

In [3]:
# Extracting links for all the pages (1 to 158) of boomlive fake news section
fakearticle_links = []
for i in range(1, 159):
  url = 'https://www.boomlive.in/fake-news/' + str(i)
  try:
    # this might throw an exception if something goes wrong.
    page=requests.get(url) 

    # send requests
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')  

    # Collecting all the links in a list
    for content in soup.find_all('h2', attrs={'class':'entry-title'}):
      link = content.find('a')
      fakearticle_links.append(link.get('href')) 
  
  # this describes what to do if an exception is thrown 
  except Exception as e:    
    # get the exception information
    error_type, error_obj, error_info = sys.exc_info()      
    #print the link that cause the problem
    print ('ERROR FOR LINK:',url)
    #print error info and line that threw the exception                          
    print (error_type, 'Line:', error_info.tb_lineno)
    continue
  

In [ ]:
fakearticle_links[:5]

In [13]:
len(fakearticle_links)

1893

In [6]:
fakearticle_links[1888:]

['/fake-news-police-no-5-day-week-for-banks-on-cards/',
 '/fake-news-police-whatsapp-message-islamic-state-group-fake/',
 '/fake-news-police-twitter-video-of-jet-airways-aircraft-on-fire-is-fake/',
 '/google-launches-fact-check-feature-globally-to-fight-fake-news/',
 '/fake-news-or-real-news-media-with-an-agenda/']

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

We have to modify the links so that the links actually work as we can see that the string extracted is the last part of the url!

**We have to add 'https://www.boomlive.in/fake-news' to the extracted links.**

In [8]:
# Modify the links so that it takes us to the particular website
str1 = 'https://www.boomlive.in/fake-news'
fakearticle_links = [str1+lnk for lnk in fakearticle_links]


In [9]:
fakearticle_links[6:10]

['https://www.boomlive.in/fake-news/fake-news/toi-ht-abp-news-fall-for-fake-account-impersonating-indrajit-chakraborty-9671',
 'https://www.boomlive.in/fake-news/fake-news/did-sushant-singh-rajput-conceptualise-upcoming-mobile-game-fau-g-9666',
 'https://www.boomlive.in/fake-news/fake-news/no-amartya-sen-did-not-say-pubg-ban-to-affect-indias-economy-9658',
 'https://www.boomlive.in/fake-news/fake-news/video-of-bengaluru-woman-abusing-2-year-old-grandson-viral-as-mumbai-9657']

**The links are modified and is working :)**

***Creating a dataset of all the fake articles***

In [37]:
# Create a dataset for storing the news articles
news_dataset = pd.DataFrame(fakearticle_links, columns=['URL'])

In [38]:
news_dataset.head()

,URL
0,https://www.boomlive.in/fake-news/fake-news/cr...
1,https://www.boomlive.in/fake-news/fake-news/un...
2,https://www.boomlive.in/fake-news/fake-news/cr...
3,https://www.boomlive.in/fake-news/fake-news/no...
4,https://www.boomlive.in/fake-news/fake-news/20...


In [26]:
title, text, summary, keywords, published_on, author = [], [], [], [], [], []   # Creating empty lists to store the data
for Url in fakearticle_links:
  article = Article(Url)

  #Call the download and parse methods to download information
  try:
    article.download()
    article.parse()
    article.nlp()
  except:
    pass
  
  # Scrape the contents of article
  title.append(article.title)                    # extracts the title of the article
  text.append(article.text)                      # extracts the whole text of article
  summary.append(article.summary)                # gives us a summary abou the article
  keywords.append(', '.join(article.keywords))   # the main keywords used in it
  published_on.append(article.publish_date)      # the date on which it was published
  author.append(article.authors)                 # the authors of the article

**Checking the lists created**

In [28]:
text[6]

'Several mainstream news outlets such as the Times of India, Hindustan Times and ABP News fell for a fake Twitter account impersonating actress Rhea Chakraborty\'s father Indrajit Chakraborty to falsely claim that the latter was tweeting following his daughter\'s arrest.\n\nThe Narcotics Control Bureau on Tuesday arrested Rhea Chakraborty for allegedly procuring drugs for late actor Sushant Singh Rajput. The NCB is investigating the drug links in Rajput\'s death. The actor was found dead in his apartment on June 14, 2020 in Mumbai and the case has since been transferred to the Central Bureau of Investigation.\n\nNews outlets such as the Times of India, Hindustan Times and ABP News cited tweets from an impostor account pretending to be Indrajit Chakraborty.\n\n\n\n\n\nBelow is a screenshot of the Hindustan Times article.\n\n\n\n\n\n\n\n\n\nThe impostor account in a series of tweets blamed the judiciary and the media for the treatment meted out to his children post the death of Sushant S

In [33]:
keywords[1]

'jammu, video, sena, image, archive, post, kangana, unrelated, support, ranaut, singh, mumbai, cars, karni, viral, senas, visuals'

In [32]:
published_on[6]

datetime.datetime(2020, 9, 9, 13, 24, 20, tzinfo=tzoffset(None, 19800))

In [34]:
author[6]

['Swasti Chatterjee']

In [39]:
# Adding the columns in the fake news dataset
news_dataset['title'] = title
news_dataset['text'] = text
news_dataset['keywords'] = keywords
news_dataset['published date'] = published_on
news_dataset['author'] = author

In [41]:
# Check the first five columns of dataset created
news_dataset.head()

,URL,title,text,keywords,published date,author
0,https://www.boomlive.in/fake-news/fake-news/cr...,Cropped Image Of Asansol Municipality Signboar...,A cropped image of a signboard of Asansol Muni...,"false, image, claims, written, text, hindi, as...",2020-09-11 14:55:25+05:30,[Sk Badiruddin]
1,https://www.boomlive.in/fake-news/fake-news/un...,Unrelated Visuals Viral As Karni Sena's Show O...,Old unrelated images from different states inc...,"jammu, video, sena, image, archive, post, kang...",2020-09-11 14:37:28+05:30,[Dilip Unnikrishnan]
2,https://www.boomlive.in/fake-news/fake-news/cr...,Cropped Video Of A Multi-Faith Ceremony For Ra...,A video of a priest reading a verse from the B...,"read, jets, misleading, video, religious, shar...",2020-09-11 14:16:32+05:30,[Anmol Alphonso]
3,https://www.boomlive.in/fake-news/fake-news/no...,"No, This Is Not A BJP Leader Threatening Judge...",A video from a 2015 episode of a dance reality...,"bjp, episode, video, father, threatening, vira...",2020-09-10 21:29:18+05:30,[Anmol Alphonso]
4,https://www.boomlive.in/fake-news/fake-news/20...,2017 Video Of Teen Assaulted By A Man In Andhr...,A 2017 video of a man groping and assaulting a...,"hindus, হনদ, 2017, video, peddled, kerala, man...",2020-09-10 20:03:03+05:30,[Suhash Bhattacharya]


**Converting the dataset to a csv file**

In [42]:
news_dataset.to_csv('Fake_news.csv')

**Reading the csv file**

In [43]:
df = pd.read_csv('Fake_news.csv')

In [44]:
# Checking the last 5 rows of the csv file
df.tail(5)

,Unnamed: 0,URL,title,text,keywords,published date,author
1888,1888,https://www.boomlive.in/fake-news/fake-news-po...,Fake News Police: No 5-Day Week For Banks On C...,People queue to deposit inside a bank in Allah...,"notification, week, fake, 5day, bank, message,...",2017-04-13 16:24:42+05:30,['Jency Jacob']
1889,1889,https://www.boomlive.in/fake-news/fake-news-po...,Fake News Police: 'Firdaus We Ascend' Islamic ...,A WhatsApp message warning people about joinin...,"fake, read, report, user, group, message, user...",2017-04-12 19:52:45+05:30,['A Staff Writer']
1890,1890,https://www.boomlive.in/fake-news/fake-news-po...,Fake News Police: Twitter Video Of Jet Airways...,Screenshot of a Twitter user page\n\nThis vide...,"fake, twitter, video, airways, jet, aircraft, ...",2017-04-12 16:22:58+05:30,['A Staff Writer']
1891,1891,https://www.boomlive.in/fake-news/google-launc...,Google Takes On Fake News With New Fact Check ...,Tech major Google is the latest to tackle the ...,"fake, search, company, google, claim, users, i...",2017-04-08 11:16:09+05:30,['A Staff Writer']
1892,1892,https://www.boomlive.in/fake-news/fake-news-or...,Fake News Or Real News? Media With An Agenda,There is a sizeable shift in perception going ...,"fake, agenda, london, south, issue, world, rea...",2017-04-03 18:25:44+05:30,['Siddhartha Dubey']


**Download the csv file in local machine**

In [45]:
from google.colab import files
files.download('Fake_news.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>